In [1]:
using ITensors

# DMRG for spinless t-V on Honeycomb lattice

In [5]:
Lx, Ly = 2, 2;
t = 1.0;
nTot = Lx*Ly*2
sites = siteinds("Fermion", Lx*Ly*2; conserve_qns=true);
state = ["Emp" for n in 1:Lx*Ly*2]
for i in 1:Lx*Ly
    state[i] = "Occ" # start from half filled
end

psi0 = randomMPS(sites, state, 10);
nsweeps = 6
maxdim = [50, 100, 200, 400, 800, 800]
cutoff = [1E-12];

function site2Idx(ix, iy, iu)
    return Lx*Ly*(iu) + (ix)*Ly + iy + 1
end;

In [8]:
function eFromV(v)
    os = OpSum();
    for ix in 0:Lx-1
        for iy in 0:Ly-1
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, iy, 1)
            os += -t, "Cdag", site2Idx(ix, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx((ix+1)%Lx, iy, 1)
            os += -t, "Cdag", site2Idx((ix+1)%Lx, iy, 1), "C", site2Idx(ix, iy, 0)
            os += -t, "Cdag", site2Idx(ix, iy, 0), "C", site2Idx(ix, (iy+1)%Ly, 1)
            os += -t, "Cdag", site2Idx(ix, (iy+1)%Ly, 1), "C", site2Idx(ix, iy, 0)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx((ix+1)%Lx, iy, 1)
            os += v, "N", site2Idx(ix, iy, 0), "N", site2Idx(ix, (iy+1)%Ly, 1)
        end
    end
    H = MPO(os, sites);
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff, outputlevel = 0);
    d = fill(0.0, Lx*Ly*2)
    for j in 1:Lx*Ly*2
        orthogonalize!(psi, j)
        psidag_j = dag(prime(psi[j], "Site"))
        d[j] = scalar(psidag_j * op(sites, "N", j) * psi[j])
    end
    # println("V=", v, " E=", energy-0.375*nTot*v, "\ndensity=", d)
    println("V=", v, " E=", energy-0.375*nTot*v)
end

eFromV (generic function with 1 method)

In [9]:
for v in 0.1:0.1:2.0
    eFromV(v)
end

V=0.1 E=-6.077158208310034
V=0.2 E=-6.1588317266270245
V=0.3 E=-6.245325367750457
V=0.4 E=-6.336949272411738
V=0.5 E=-6.434013869095788
V=0.6 E=-6.536823760926662
V=0.7 E=-6.645670658480553
V=0.8 E=-6.760825601737061
V=0.9 E=-6.882530845058997
V=1.0 E=-7.010991894885804
V=1.1 E=-7.146370265492903
V=1.2 E=-7.288777530218335
V=1.3 E=-7.4382711794964695
V=1.4 E=-7.594852653628715
V=1.5 E=-7.758467715923196
V=1.6 E=-7.929009103680209
V=1.7 E=-8.106321179900274
V=1.8 E=-8.29020614289281
V=1.9 E=-8.480431256636262
V=2.0 E=-8.676736547233341


# DMRG for spinless t-V on Square lattice

In [16]:
Lx, Ly = 4, 6;
t = 1.0;
sites = siteinds("Fermion", Lx*Ly; conserve_qns=true);
state = ["Emp" for n in 1:Lx*Ly]
for i in 1:Int(Lx*Ly/2)
    state[i] = "Occ" # start from half filled
end

psi0 = randomMPS(sites, state, 10);
nsweeps = 6
maxdim = [50, 100, 200, 400, 800, 800]
cutoff = [1E-12];

function site2Idx(ix, iy)
    return (ix)*Ly + iy + 1
end;

In [17]:
function eFromV(v)
    os = OpSum();
    for ix in 0:Lx-1
        for iy in 0:Ly-1
            i, j1, j2 = site2Idx(ix, iy), site2Idx((ix+1)%Lx, iy), site2Idx(ix, (iy+1)%Ly)
            
            os += -t, "Cdag", i, "C", j1
            os += -t, "Cdag", j1, "C", i
            os += -t, "Cdag", i, "C", j2
            os += -t, "Cdag", j2, "C", i

            os += v, "N", i, "N", j1
            os += v, "N", i, "N", j2
        end
    end
    H = MPO(os, sites);
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff, outputlevel = 0);
    d = fill(0.0, Lx*Ly*2)
    for j in 1:Lx*Ly
        orthogonalize!(psi, j)
        psidag_j = dag(prime(psi[j], "Site"))
        d[j] = scalar(psidag_j * op(sites, "N", j) * psi[j])
    end
    # println("V=", v, " E=", energy-0.375*nTot*v, "\ndensity=", d)
    println("V=", v, " E=", energy-0.5*Lx*Ly*v)
end

eFromV (generic function with 1 method)

In [18]:
for v in 0.1:0.1:2.0
    eFromV(v)
end

V=0.1 E=-20.22322224663647
V=0.2 E=-20.4603335041311
V=0.3 E=-20.71270344687231
V=0.4 E=-20.98112574374815
V=0.5 E=-21.272172070409923
V=0.6 E=-21.587368572121747
V=0.7 E=-21.934715820191105
V=0.8 E=-22.324041043367636
V=0.9 E=-22.764569888123837
V=1.0 E=-23.266888663313026
V=1.1 E=-23.829263540781007
V=1.2 E=-24.45346444831486
V=1.3 E=-25.136660992277136
V=1.4 E=-25.86371631180001
V=1.5 E=-26.635249840573902
V=1.6 E=-27.44153925336677
V=1.7 E=-28.282458379124684
V=1.8 E=-29.150132974020753
V=1.9 E=-30.044055867998246
V=2.0 E=-30.960930985608186
